In [1]:
import os
import numpy as np 
import damask

In [2]:
%config Completer.use_jedi = False

In [3]:
# some pre-requisites, after `source env/DAMASK.sh`
damask_root = %env DAMASK_ROOT

In [4]:
dire = 'test_mat'
fn_vti = 'g.vti'
fn_load = 'l.yaml'
fn_material = 'm.yaml'
if not os.path.exists(dire): os.mkdir(dire)

# extreme simple geometry and load files

In [5]:
# geometry file (3d periodic grid)
cells = [2,2,1]
grid = damask.GeomGrid(material=np.zeros(cells),size=cells)
grid.save(f'{dire}/{fn_vti}')

In [6]:
# load file - 2 load incrs of tensile test
load_str = """solver: {mechanical: spectral_basic}
loadstep:
- boundary_conditions:
    mechanical:
      dot_F:
      - [0.001, 0, 0]
      - [0, x, 0]
      - [0, 0, x]
      P:
      - [x, x, x]
      - [x, 0, x]
      - [x, x, 0]
  discretization: {t: 0.5, N: 2}
  f_out: 1"""
with open(f'{dire}/{fn_load}', 'w') as fw:
    fw.write(load_str)

# material file based on example materials, single phase for testing

In [7]:
# PLEASE CHANGE ACCORDING TO YOUR PROBLEM !!!!
phase_info_yml = 'examples/config/phase/Mg.yaml'
phase_elast_yml = 'examples/config/phase/mechanical/elastic/Hooke_Mg.yaml'

In [8]:
mat_str = """homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}
material:
- constituents:
  - phase: test
    v: 1.0
    O: [1.0, 0.0, 0.0, 0.0]
  homogenization: SX
phase: 
  test"""

mat = damask.YAML(mat_str)

In [9]:
test_mat = damask.YAML.load(os.path.join(damask_root, phase_info_yml)) 
test_mat_el = damask.YAML.load(os.path.join(damask_root, phase_elast_yml))
test_mat.update(mechanical={'elastic':test_mat_el})

In [10]:
# del unused key and val
test_mat.pop('rho')

1740.0

In [11]:
mat.update(phase={'test':test_mat})
mat.save(f'{dire}/{fn_material}')

# test run

In [12]:
%%time
os.environ['OMP_NUM_THREADS'] = '1'
run_log, err_log = damask.util.run(f'DAMASK_grid -w {dire} -g {fn_vti} -l {fn_load}  -m {fn_material}')

running 'DAMASK_grid -w test_mat -g g.vti -l l.yaml  -m m.yaml' in './'
CPU times: user 2.8 ms, sys: 1.65 ms, total: 4.46 ms
Wall time: 393 ms
